In [1]:
# 데이터 분석을 위한 pandas, 수치계산을 위한 numpy, 시각화를 위한 seaborn, matplotlib 을 로드합니다.
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df_train = pd.read_csv('C:/Users/KimDongyoung/Desktop/GBT해커톤/GBT_hackerton/data/train_df_1009_v2.csv')
df_test = pd.read_csv('C:/Users/KimDongyoung/Desktop/GBT해커톤/GBT_hackerton/data/test_df_1009_v2.csv')
submission = pd.read_csv('C:/Users/KimDongyoung/Desktop/GBT해커톤/GBT_hackerton/data/sample_submission.csv')

In [3]:
df_train.columns

Index(['ID', '분류', '제목', '키워드'], dtype='object')

In [6]:
# df_test에 '분류' 컬럼을 추가하고, 값을 None으로 채웁니다.
df_test['분류'] = None

# df_train과 df_test를 하나의 데이터프레임으로 합칩니다.
df_combined = pd.concat([df_train, df_test], ignore_index=True)

# 합쳐진 데이터프레임을 출력합니다.
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77719 entries, 0 to 77718
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      77719 non-null  object
 1   분류      54314 non-null  object
 2   제목      77719 non-null  object
 3   키워드     77719 non-null  object
dtypes: object(4)
memory usage: 2.4+ MB


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer 

## TD-IDF 벡터화

In [8]:
tfidfv = TfidfVectorizer()

tfidfv.fit(df_combined['키워드'])

TfidfVectorizer()

In [9]:
# 변환
vectorized = tfidfv.transform(df_combined['키워드'])

In [10]:
vectorized.shape

(77719, 237607)

In [11]:
# 단어 사전
feature_names = tfidfv.get_feature_names_out()
print(len(feature_names))
feature_names[:10]

237607


array(['00065', '000가', '000페소', '000필지', '00446', '007작전', '00군', '00동',
       '00유치원', '00특례시'], dtype=object)

In [12]:
# np.sum 으로 위에서 구한 train_feature_vector 의 값을 모두 더합니다. axis=0 으로 합니다. 
dist = np.sum(vectorized, axis=0)

vocab_count = pd.DataFrame(dist, columns=feature_names)
vocab_count

,00065,000가,000페소,000필지,00446,007작전,00군,00동,00유치원,00특례시,...,李之浩,李健熙,李元和,李壽興,梨花雨,臨場,臨場記,立春,立春大吉,糖醋肉
0,0.256942,0.064629,0.070212,0.12509,0.055694,0.05394,0.261966,0.109424,0.04756,0.113524,...,0.025188,0.048147,0.044,0.035206,0.092278,0.118992,0.118992,0.357906,0.32638,0.027016


In [13]:
vocab_count.T[0].sort_values(ascending=False).head()

반도체    1738.696425
사업     1531.877261
조성     1020.052926
도시      951.365641
확진자     943.665750
Name: 0, dtype: float64

## 벡터화 데이터를 다시 train, test로 나눠준다

In [14]:
vectorized_train_x = vectorized[:len(df_train)]
vectorized_test_x = vectorized[len(df_train):]

## target 값 레이블 인코딩

In [15]:
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the '키워드' column
df_train['키워드_encoded'] = label_encoder.fit_transform(df_train['키워드'])

# Display the first few rows to verify
df_train[['키워드', '키워드_encoded']].head()

,키워드,키워드_encoded
0,용인문화재단 인문학 콘서트 뮤지컬 이야기 개최 인문학 콘서트 뮤지컬 이야기 용인문화...,30184
1,농촌 테마파크 단체 체험객 체험료 체험일 기준 용인시통합예약사이트 접수 용인특례시 ...,10424
2,노후 주택 에너지 성능 개선 신청 연장 용인시청 노후 건축물 환경친화적 개선 온실 ...,10109
3,수원 고양시 특례시 지정 도시경쟁력 증가 도내 인구 수원 고양 특례시 지정 골자 지...,23394
4,스페인 미국 국제 명예 자문관 위촉 역할 대외 홍보 권태면 주코스타리카 대사 이정관...,24379


In [16]:
# y_train 정의
y_train = df_train['키워드_encoded']

## 로지스틱 회귀

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, f1_score, log_loss

In [18]:
# 학습 데이터와 검증 데이터를 나눕니다.
X_train, X_val, y_train_split, y_val = train_test_split(vectorized_train_x, y_train, test_size=0.2, random_state=42)

# 로지스틱 회귀 모델을 초기화합니다.
model = LogisticRegression(max_iter=1000)

# 모델을 학습시킵니다.
model.fit(X_train, y_train_split)

# 검증 데이터로 예측을 수행합니다.
y_val_pred = model.predict(X_val)

# 검증 데이터에 대한 macro F1 score를 계산합니다.
macro_f1 = f1_score(y_val, y_val_pred, average='macro')

# 검증 데이터에 대한 예측 확률을 계산합니다.
y_val_pred_proba = model.predict_proba(X_val)

# 검증 데이터에 대한 log loss를 계산합니다.
loss = log_loss(y_val, y_val_pred_proba)

# 결과를 출력합니다.
print(f'Macro F1 Score: {macro_f1:.4f}')
print(f'Log Loss: {loss:.4f}')

MemoryError: Unable to allocate 76.7 GiB for an array with shape (43351, 237608) and data type float64

In [ ]:
# 테스트 데이터에 대한 예측을 수행합니다.
y_test_pred = model.predict(vectorized_test_x)

# 예측 결과를 출력합니다.
print(y_test_pred)